# 目的
kerasでのssplit実験に向けた動作テスト用作業ノート。


In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding, Reshape,Convolution1D
from keras.utils import np_utils
from keras import backend as K
import numpy as np

Using TensorFlow backend.


In [54]:
model = Sequential()
model.add(Embedding(10000, 4, input_length=3))
model.add(Convolution1D(2, 3, border_mode='same', input_shape=(4, 3)))
model.add(Convolution1D(3, 1, border_mode='same', input_shape=(2, 3)))
model.add(Activation('relu'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [58]:
input_array = np.random.randint(10,size=(2,3))
gold_array = np.random.randint(3,size=(2,3))
output_array = model.predict(input_array)

In [63]:
input_array.reshape(2,3,1)

array([[[7],
        [2],
        [0]],

       [[7],
        [8],
        [2]]])

In [64]:
gold_array.reshape(2,3,1)

array([[[0],
        [0],
        [0]],

       [[0],
        [0],
        [1]]])

In [65]:
output_array

array([[[ 0.02129532,  0.01256803,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.02512304,  0.01925107]],

       [[ 0.01018128,  0.01088626,  0.00229398],
        [ 0.00980728,  0.        ,  0.        ],
        [ 0.        ,  0.00454086,  0.0075867 ]]], dtype=float32)

In [66]:
model.fit(input_array, gold_array, nb_epoch=1)

Exception: Error when checking model target: expected activation_1 to have 3 dimensions, but got array with shape (2, 3)